# Builds for Gypsy mod

Developed for gypsy 1.35.6

In [ ]:
import yaml
from collections import Counter, defaultdict
import itertools
from math import comb
from tqdm.notebook import tqdm
import random
from functools import lru_cache

## Load data

In [ ]:
IDEAS_PATHS = ['data/gypsy_transformed/ideas.yaml', 'data/gypsy_transformed/ideas_flogi.yaml', 'data/gypsy_transformed/ideas_flogi_relig.yaml']

class Idea():
    def __init__(self, name:str, type: str, effect: dict):
        self.name = name
        self.type = type
        self.effect = effect

    def __repr__(self):
        return f'Policy({self.name}, {self.type}, {self.effect})'

IDEAS = {}

for path in IDEAS_PATHS:
    with open(path, 'r') as f:
        ideas_dict = yaml.load(f, Loader=yaml.FullLoader)
    for name, value in ideas_dict.items():
        category = value['category']
        effect = Counter()
        for key in value.keys():
            if key != 'category' and key != 'important':
                effect_set = value[key]
                for k, v in effect_set.items():
                    effect[k] += abs(v)
        IDEAS[name] = Idea(name=name, type=category, effect=effect)


In [ ]:
POLICIES_PATH = 'data/gypsy_transformed/policies.yaml'

class Policy():
    def __init__(self, name:str, type: str, req: tuple, effect: dict):
        self.name = name
        self.type = type
        self.req = req
        self.effect = effect

    def __repr__(self):
        return f"Policy({self.name}, {self.type}, {self.req}, {self.effect})"

with open(POLICIES_PATH, 'r') as f:
    policies_dict = yaml.load(f, Loader=yaml.FullLoader)
POLICIES = []
skip_keys = {'monarch_power', 'potential', 'allow', 'ai_will_do'}
for name, value in policies_dict.items():
    monarch_power = value['monarch_power']
    allow_A = {value['allow'][0]} if isinstance(value['allow'][0], str) else set(value['allow'][0]['OR'])
    allow_B = {value['allow'][1]} if isinstance(value['allow'][1], str) else set(value['allow'][1]['OR'])
    effect = Counter()
    for key in value.keys():
        if key not in skip_keys:
            effect[key] = abs(value[key])
    POLICIES.append(Policy(
            name = name, 
            type = value['monarch_power'], 
            req = (allow_A, allow_B),
            effect = effect
        ))


In [ ]:
class Build():
    def __init__(self, 
                 ideas: tuple[str], 
                 score: float, 
                 total_effect: Counter, 
                 war_policies_effect: dict
    ):
        self.ideas = ideas
        self.score = score
        self.total_effect = total_effect
        self.war_policies_effect = war_policies_effect

    def __repr__(self):
        return f"Build( {self.score:.2f} - ideas: {self.ideas})"
    
def print_build(build: Build, effects: list[str]):
    print('--------------------------------------------------------------------')
    print(f"Build - score: {build.score} - ideas: {build.ideas}")
    for effect in effects:
        print(f"\t{effect}: {build.total_effect[effect]:.2f}")

## Build rules

In [ ]:
admin_idea_names = [v.name for k, v in IDEAS.items() if v.type == 'ADM']
diplo_idea_names = [v.name for k, v in IDEAS.items() if v.type == 'DIP']
military_idea_names = [v.name for k, v in IDEAS.items() if v.type == 'MIL']

In [ ]:
admin_idea_allowed = [
    'innovativeness_ideas',
    'economic_ideas',
    'expansion_ideas',
    'administrative_ideas',
    'humanist_ideas',
    'judiciary',
    'development',
    'strong_men',
    # 'fem_boy', # Commented since the same as strong_men
    # 'public_admin',
    'centralism',
    'decentralism',
    # Uncomment your government type
    'monarchie0',
    # 'republik0',
    # 'aristo0',
    # 'diktatur0',
    # 'horde0',
    # Uncomment your religion
    'religious_ideas',
    # 'katholisch0',
    # 'protestant0',
    # 'reformiert0',
    # 'orthodox0',
    # 'islam0',
    # 'tengri0',
    # 'hindu0',
    # 'confuci0',
    # 'budda0',
    # 'norse0',
    # 'shinto0',
    # 'cathar0',
    # 'coptic0',
    # 'romuva0',
    # 'suomi0',
    # 'jew0',
    # 'slav0',
    # 'helle0',
    # 'mane0',
    # 'animist0',
    # 'feti0',
    # 'zoro0',
    # 'ancli0',
    # 'nahu0',
    # 'mesoam0',
    # 'inti0',
    # 'tote0',
    # 'shia0',
    # 'ibadi0',
    # 'hussite0',
    # 'alche0'
]
admin_not_compatible = [
    ('strong_men', 'fem_boy',),
    ('centralism', 'decentralism',),
]

In [ ]:
diplo_idea_allowed = [
    'spy_ideas',
    'dynastic',
    'influence_ideas',
    'trade_ideas',
    'exploration_ideas',
    'maritime_ideas',
    'heavy_ship',
    'galley_ship',
    'trade_ship',
    'colonial_emp',
    'assimilation',
    'sociaty',
    'propaganda',
    'fleet_base',
    'nationalismus',
    'konigreich0',
    'imperialismus'
]
dip_not_compatible = [
    ('heavy_ship', 'galley_ship'),
    ('heavy_ship', 'trade_ship'),
    ('galley_ship', 'trade_ship')
]

In [ ]:
military_idea_allowed = [
    'offensive',
    'defensive',
    'quality',
    'quantity',
    'general_staff',
    'standing_army',
    'conscription',
    'merc_army',
    'weapon_quality',
    'fortress',
    'war_production',
    'formation0',
    'militarism',
    'shock_ideas',
    'fire_ideas'
 ]
mil_not_compatible = [
    ('offensive', 'defensive'),
    ('quality', 'quantity'),
    ('standing_army', 'conscription'),
    ('shock_ideas', 'fire_ideas')
]

## Build score

In [ ]:
# Country weights are used to determine the score of a country during peace time
COUNTRY_WEIGHTS = {
    'estate_nationalist': 10,               # Having this estate is 10 point
    'development_cost': 150,               # Each 10% is 15 points
    'free_policy': 5,                       # Each free extra policy is 5 points
    'free_adm_policy': 2,                   # Each free extra policy is 2 points
    'free_dip_policy': 2,                   # Each free extra policy is 2 points
    'free_mil_policy': 1,                   # Each free extra policy is 1 points
    'possible_policy': 12,                  # Each extra policy column is 12 points
    'possible_adm_policy': 4,               # Each possible policy is 4 points
    'possible_dip_policy':  4,              # Each possible policy is 4 points
    'possible_mil_policy': 4,               # Each possible policy is 4 points
    'technology_cost': 60,                 # Each 10% discount is 6 points
    'adm_tech_cost_modifier': 20,          # Each 10% discount is 2 point
    'dip_tech_cost_modifier':  20,         # Each 10% discount is 2 point
    'mil_tech_cost_modifier':  20,         # Each 10% discount is 2 point
    'idea_cost': 60,                       # Each 10% discount is 6 points
    'governing_capacity_modifier': 30,      # Each 10% is 3 points
    'advisor_cost': 10,                    # Each 10% discount is 1 point
    'global_tax_modifier': 5,               # Each 10% is 0.5 point
    'production_efficiency': 5,             # Each 10% is 0.5 point
    'global_trade_goods_size_modifier': 5,  # Each 10% is 0.5 point
    'trade_efficiency': 5,                  # Each 10% is 0.5 point
    'state_maintenance_modifier': 5,       # Each 10% discount is 0.5 point
}

# War weights are used to determine the score of a country during war time and to find the best policies
MILITARY_WEIGHTS ={
    'infantry_power': 150,                  # Each 10% is 15 points
    'cavalry_power': 150,                   # Each 10% is 15 points
    'artillery_power': 150,                 # Each 10% is 15 points
    'discipline': 300,                      # Each 10% is 30 points
    'fire_damage': 150,                     # Each 10% is 15 points
    'fire_damage_received': 150,           # Each 10% is 15 points
    'shock_damage': 150,                    # Each 10% is 15 points
    'shock_damage_received': 150,          # Each 10% is 15 points
    'land_morale': 150,                     # Each 10% is 15 points
    'global_manpower_modifier': 50,         # Each 10% is 5 points
    'land_forcelimit_modifier': 50,         # Each 10% is 5 points
    'land_maintenance_modifier': 20        # Each 10% discount is 2 points
}

def score(effects: dict, weights: list[dict], debug=False):
    score = 0
    for weight in weights:
        for key, value in effects.items():
            if key in weight:
                if debug:
                    print(f'{key}: {value} -> {value * weight[key]}')
                score += value * weight[key]
    return score

## Support

In [ ]:
@lru_cache()
def get_available_policies(ideas: tuple[str]):
    policy_pool = []
    for policy in POLICIES:
       for idea_A, idea_B in itertools.combinations(ideas, 2):
            if idea_A in policy.req[0] and idea_B in policy.req[1]:
                policy_pool.append(policy)
            elif idea_A in policy.req[1] and idea_B in policy.req[0]:
                policy_pool.append(policy)
    policies_pool_adm = [p for p in policy_pool if p.type == 'ADM']
    policies_pool_dip = [p for p in policy_pool if p.type == 'DIP']
    policies_pool_mil = [p for p in policy_pool if p.type == 'MIL']
    return policies_pool_adm, policies_pool_dip, policies_pool_mil


def get_max_policy_slots(ideas_effect: Counter, base=4):
    adm_max_policy = base + ideas_effect['possible_policy'] + ideas_effect['possible_adm_policy']
    dip_max_policy = base + ideas_effect['possible_policy'] + ideas_effect['possible_dip_policy']
    mil_max_policy = base + ideas_effect['possible_policy'] + ideas_effect['possible_mil_policy']
    return adm_max_policy, dip_max_policy, mil_max_policy

In [ ]:
def get_ideas_effect(ideas: tuple[str]):
    total_effect = Counter()
    for idea in ideas:
        total_effect.update(IDEAS[idea].effect)
        
    return total_effect


def get_micro_management_policies_effect(ideas: tuple[str], adm_max_policy=4, dip_max_policy=4, mil_max_policy=4):
    def get_max_effect(policies: list[Policy], effects: set[str], max_policies: int):
        effect_policies = [max(p.effect.get(effect, 0) for effect in effects) for p in policies if any(e in p.effect for e in effects)]
        effect_policies.sort(reverse=True)
        return sum(effect_policies[:max_policies])


    total_effect = Counter()
    available_adm_policies, available_dip_policies, available_mil_policies = get_available_policies(ideas)

    # Potential dev_cost policies 
    policy_dev_cost = 0
    dev_effects = {'development_cost'}
    policy_dev_cost += get_max_effect(available_adm_policies, dev_effects, adm_max_policy)
    policy_dev_cost += get_max_effect(available_dip_policies, dev_effects, dip_max_policy)
    policy_dev_cost += get_max_effect(available_mil_policies, dev_effects, mil_max_policy)
    total_effect['development_cost'] += policy_dev_cost

    # Potential tech cost policies
    adm_tech_cost_modifier = 0
    adm_tech_effects = {'technology_cost', 'adm_tech_cost_modifier'} 
    adm_tech_cost_modifier += get_max_effect(available_adm_policies, adm_tech_effects, adm_max_policy)
    adm_tech_cost_modifier += get_max_effect(available_dip_policies, adm_tech_effects, dip_max_policy)
    adm_tech_cost_modifier += get_max_effect(available_mil_policies, adm_tech_effects, mil_max_policy)
    total_effect['adm_tech_cost_modifier'] += adm_tech_cost_modifier
    
    dip_tech_cost_modifier = 0
    dip_tech_effects = {'technology_cost', 'dip_tech_cost_modifier'}
    dip_tech_cost_modifier += get_max_effect(available_adm_policies, dip_tech_effects, adm_max_policy)
    dip_tech_cost_modifier += get_max_effect(available_dip_policies, dip_tech_effects, dip_max_policy)
    dip_tech_cost_modifier += get_max_effect(available_mil_policies, dip_tech_effects, mil_max_policy)
    total_effect['dip_tech_cost_modifier'] += dip_tech_cost_modifier

    mil_tech_cost_modifier = 0
    mil_tech_effects = {'technology_cost', 'mil_tech_cost_modifier'}
    mil_tech_cost_modifier += get_max_effect(available_adm_policies, mil_tech_effects, adm_max_policy)
    mil_tech_cost_modifier += get_max_effect(available_dip_policies, mil_tech_effects, dip_max_policy)
    mil_tech_cost_modifier += get_max_effect(available_mil_policies, mil_tech_effects, mil_max_policy)
    total_effect['mil_tech_cost_modifier'] += mil_tech_cost_modifier

    # Potential idea cost policies
    policy_idea_cost = 0
    idea_effects = {'idea_cost'}
    policy_idea_cost += get_max_effect(available_adm_policies, idea_effects, adm_max_policy)
    policy_idea_cost += get_max_effect(available_dip_policies, idea_effects, dip_max_policy)
    policy_idea_cost += get_max_effect(available_mil_policies, idea_effects, mil_max_policy)
    total_effect['idea_cost'] += policy_idea_cost

    return total_effect


def get_war_policies_effect(ideas: tuple[str], war_weights: dict, adm_max_policy=4, dip_max_policy=4, mil_max_policy=4):
    total_effect = Counter()

    total_effect = {}
    available_adm_policies, available_dip_policies, available_mil_policies = get_available_policies(ideas)
    
    # ADM policy slots
    adm_war_policies = [(score(p.effect, [war_weights]), p) for p in available_adm_policies]
    adm_war_policies.sort(key=lambda x: x[0], reverse=True)
    adm_war_policies = [p for p in adm_war_policies if p[0] > 0]
    for p in adm_war_policies[:min(adm_max_policy, len(adm_war_policies))]:
        total_effect.update(p[1].effect)

    # DIP war policies
    dip_war_policies = [(score(p.effect, [war_weights]), p) for p in available_dip_policies]
    dip_war_policies.sort(key=lambda x: x[0], reverse=True)
    dip_war_policies = [p for p in dip_war_policies if p[0] > 0]
    for p in dip_war_policies[:min(dip_max_policy, len(dip_war_policies))]:
        total_effect.update(p[1].effect)

    # MIL war policies
    mil_war_policies = [(score(p.effect, [war_weights]), p) for p in available_mil_policies]
    mil_war_policies.sort(key=lambda x: x[0], reverse=True)
    mil_war_policies = [p for p in mil_war_policies if p[0] > 0]
    for p in mil_war_policies[:min(mil_max_policy, len(mil_war_policies))]:
        total_effect.update(p[1].effect)

    return total_effect


In [ ]:
def compute_build(ideas: tuple[str], base_policy_slots=4, debug=False) -> Build:
    # TODO: implement caching
 
    ideas_effect = get_ideas_effect(ideas)
    adm_max_policy, dip_max_policy, mil_max_policy = get_max_policy_slots(ideas_effect, base_policy_slots)
    micro_management_policies_effect = get_micro_management_policies_effect(ideas, adm_max_policy, dip_max_policy, mil_max_policy)
    war_policies_effect = get_war_policies_effect(ideas, MILITARY_WEIGHTS, adm_max_policy, dip_max_policy, mil_max_policy)

    total_effect = Counter()
    total_effect.update(ideas_effect)
    total_effect.update(micro_management_policies_effect)
    total_effect.update(war_policies_effect)


    
    return Build(
        ideas=ideas, 
        score=score(total_effect, [COUNTRY_WEIGHTS, MILITARY_WEIGHTS], debug=debug),
        total_effect=total_effect,
        war_policies_effect=war_policies_effect,
    )

# print(compute_build(('strong_men', 'weapon_quality'), debug=True))
# print(compute_build(('innovativeness_ideas', 'spy_ideas', 'offensive'), debug=True))

## Find the best build

In [ ]:
builds = defaultdict(dict)

IDEA_COUNT_THRESHOLD = 0.39

def expand_idea_set(idea_set: set):
    idea_set_count = len(idea_set)
    adm_idea_count = len([x for x in idea_set if x in admin_idea_names])
    dip_idea_count = len([x for x in idea_set if x in diplo_idea_names])
    mil_idea_count = len([x for x in idea_set if x in military_idea_names])
    if adm_idea_count / idea_set_count < IDEA_COUNT_THRESHOLD:
        for idea in admin_idea_allowed:
            if idea not in idea_set:
                expanded_idea_set = idea_set.union({idea})
                for rule_A, rule_B in admin_not_compatible:
                    if rule_A in expanded_idea_set and rule_B in expanded_idea_set:
                        break
                else:
                    yield expanded_idea_set
    if dip_idea_count / idea_set_count < IDEA_COUNT_THRESHOLD:
        for idea in diplo_idea_allowed:
            if idea not in idea_set:
                expanded_idea_set = idea_set.union({idea})
                for rule_A, rule_B in dip_not_compatible:
                    if rule_A in expanded_idea_set and rule_B in expanded_idea_set:
                        break
                else:
                    yield expanded_idea_set
    if mil_idea_count / idea_set_count < IDEA_COUNT_THRESHOLD:
        for idea in military_idea_allowed:
            if idea not in idea_set:
                expanded_idea_set = idea_set.union({idea})
                for rule_A, rule_B in mil_not_compatible:
                    if rule_A in expanded_idea_set and rule_B in expanded_idea_set:
                        break
                else:
                    yield expanded_idea_set

## 3-policy builds

In [ ]:
total_options = len(admin_idea_allowed) *  len(diplo_idea_allowed) * len(military_idea_allowed)

with tqdm(total=total_options) as pbar:
    for admin_ideas in admin_idea_allowed:
        for diplo_ideas in diplo_idea_allowed:
            for military_ideas in military_idea_allowed:
                idea_list = [admin_ideas,  diplo_ideas,  military_ideas]
                idea_list.sort()
                ideas = tuple(idea_list)
                build = compute_build(ideas)
                builds[3][ideas] = (build)
                pbar.update(1)

## 4-policy builds

In [ ]:
for ideas in tqdm(builds[3].keys(), total=len(builds[3])):
    for new_idea_set in expand_idea_set(set(ideas)):
        idea_list = list(new_idea_set)
        idea_list.sort()
        new_ideas = tuple(idea_list)
        new_build = compute_build(new_ideas)
        builds[4][new_ideas] = new_build

## 5-policy builds

In [ ]:
for ideas in tqdm(builds[4].keys(), total=len(builds[4])):
    for new_idea_set in expand_idea_set(set(ideas)):
        idea_list = list(new_idea_set)
        idea_list.sort()
        new_ideas = tuple(idea_list)
        new_build = compute_build(new_ideas)
        builds[5][new_ideas] = new_build

## 6-policy builds

In [ ]:
total_options = comb(len(admin_idea_allowed), 2) * comb(len(diplo_idea_allowed), 2) * comb(len(military_idea_allowed), 2)

with tqdm(total=total_options) as pbar:
    for admin_ideas in itertools.combinations(admin_idea_allowed, 2):
        for diplo_ideas in itertools.combinations(diplo_idea_allowed, 2):
            for military_ideas in itertools.combinations(military_idea_allowed, 2):
                idea_set = set(admin_ideas + diplo_ideas + military_ideas)
                for rule_A, rule_B in admin_not_compatible + dip_not_compatible + mil_not_compatible:
                    if rule_A in idea_set and rule_B in idea_set:
                        break
                else:
                    idea_list = list(idea_set)
                    idea_list.sort()
                    ideas = tuple(idea_list)
                    build = compute_build(ideas)
                    builds[6][ideas] = build
                pbar.update(1)


## Expand on the best builds

In [ ]:
EXPAND_BEST_N = 1000
EXPAND_RANDOM_N = 3000

def get_ideas_to_expand(build_list: list[Build], best_n=EXPAND_BEST_N, random_n=EXPAND_RANDOM_N):
    build_list.sort(key=lambda x: x.score, reverse=True)
    best_builds = build_list[:best_n]
    random_builds = random.sample(build_list[best_n:], random_n)
    return best_builds + random_builds


In [ ]:
for i in range(6, 13):
    builds_to_expand = get_ideas_to_expand(list(builds[i].values()))
    for build in tqdm(builds_to_expand, desc=f'Expanding builds {i}'):
        for new_idea_set in expand_idea_set(set(build.ideas)):
            idea_list = list(new_idea_set)
            idea_list.sort()
            new_ideas = tuple(idea_list)
            new_build = compute_build(new_ideas)
            builds[i+1][new_ideas] = new_build